In [2]:
import csv
import json
import sqlite3
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from datetime import datetime

In [1]:
from google.colab import drive
drive.mount('/content/drive')
ROOT='''/content/drive/MyDrive/Colab Notebooks/googledrive/objectDetection'''

Mounted at /content/drive


In [3]:
def plot_learning_curve_single_subject(marks_conf,counterbalance):

    fig, ax = plt.subplots(figsize=(5, 3))
    rect = patches.Rectangle((0,-0.2), 3, 1.4, linewidth=1,  facecolor='yellow', alpha=0.5)#edgecolor='r',
    conf_mat = marks_conf[9]
    marks_acc = [(conf_mat.diagonal() / (conf_mat.sum(1)+1e-7)).mean()]
    for i in range(1,6):
        conf_mat = marks_conf[(i+1)*8-1] - marks_conf[i*8-1]
        acc = (conf_mat.diagonal() / (conf_mat.sum(1)+1e-7)).mean()
        marks_acc.append(acc)

    ax.scatter(range(len(marks_acc)),marks_acc,c='black',s=60)

    for i in range(len(marks_acc)-1):
        ax.plot([i,i+1],[marks_acc[i],marks_acc[i+1]],c='black')
    plt.yticks([_*0.1 for _ in range(0,11,2)],[round(_*0.1,1) for _ in range(0,11,2)],fontsize=20)
    plt.xticks(range(len(marks_acc)),[_*8 for _ in range(1,len(marks_acc)+1)],fontsize=20)
    plt.suptitle('counterbalance: {}'.format(counterbalance), fontsize=20)
    # Add the patch to the Axes
    ax.add_patch(rect)
    ax.xaxis.set_ticks_position('none')
    ax.yaxis.set_ticks_position('none')
    ax.spines[['right', 'left']].set_visible(False)

def construct_confmat():
    labels = ['h_azure','h_detectron','h_google','h_rekognition','h_chatgpt']#,'h_catch']
    confmatrices = {**{'h_machine':np.zeros((2,2))}, **{label: np.zeros((1,2)) for label in labels}}
    return confmatrices

def print_metadata(data,dummy_mark='NA'):
    print("====="*20)
    datetime_object1 = datetime.strptime(data[11].split('.')[0], '%Y-%m-%d %H:%M:%S')
    datetime_object2 = datetime.strptime(data[13].split('.')[0], '%Y-%m-%d %H:%M:%S')
    time_used = datetime_object2 - datetime_object1
    time_used = round(time_used.seconds/60,1)
    print("[Finished Date]",datetime_object2)
    print("[Time Used] {} mins".format(time_used))
    print("[Questionnaire]", json.loads(data[-1])["questiondata"])
    print("[Counterbalance]", json.loads(data[-1])["counterbalance"])
    print("[Dummy Test Acc]", dummy_mark)
    return time_used, json.loads(data[-1])["counterbalance"]

def get_gt_from_pagename(pagename):
    # example:  'page_name': '568_e_0.html/',
    return {'a':'Human','b':'Machine','c':'Machine','d':'Machine','e':'Machine','f':'Machine'}[pagename.split('_')[1]]

def get_model_from_pagename(pagename):
    # example:  'page_name': '568_e_0.html/',
    return {'b':'azure','c':'detectron','d':'google','e':'rekognition','f':'chatgpt'}[pagename.split('_')[1]]

conn = sqlite3.connect(ROOT+"/ObjDet_ChatGPT/ec2_db/turing_object_chatgpt.db")
cursor = conn.cursor()
cursor.execute("select * from tablechairDB;")

label_dict = {'human':0,'machine':1,'Human':0,'Machine':1,'azure':1,'detectron':1,'google':1,'catch':1,'rekognition':1,'chatgpt':1}
oneWord_curr_d = {'0':True,'1':False} # dummy test gt
topic_d = {'1':True,'2':False} # user response to dummy tests
modelname_dict = {'catch':'CatchTrial','azure':'Azure','detectron':'Detectron','rekognition':'Rekognition','google':'Google','chatgpt':'ChatGPT'}


count = 0
participant_count = 0
trials_dict = {}
all_data = cursor.fetchall()
completed_counterbalances = []
marker_overall = []

confmat_overall = construct_confmat()
catch_trial_mat_overall = np.zeros((1,2))
swap_trial_mat_overall = np.zeros((1,2))
for i in range(len(all_data)):
    if all_data[i][15] in [5]:
#         print(all_data[i][11])
        data_dict = json.loads(all_data[i][17])
        if data_dict['mode'] != 'live':
            continue
        # filter out subjects using dummy trial accuracy
        # original threshold in the paper: 7/12
        dummy_mark = 0
        nTrials = 50
        threshold = nTrials*(7/12) # change to number of total trials
        for dat in data_dict['data']:

            if dat['trialdata']['phase'] == 'TEST':
                present = dat['trialdata']['oneWord_curr'] == '0'
                response_present = dat['trialdata']['topic'] == '1'
                if present == response_present:
                    dummy_mark += 1
        if dummy_mark < threshold:
#             print(dummy_mark)
            continue
        marker_overall.append(dummy_mark)

        # construct conf matrices
        time_used, counterbalance = print_metadata(all_data[i],"{}/{}".format(dummy_mark,nTrials))
        completed_counterbalances.append(counterbalance)
#         marks = []
#         marks_conf = []
        confmatrices = construct_confmat() # without catch
        catch_trial_mat = np.zeros((1,2))
        swap_trial_mat = np.zeros((1,2))
        marks = []
        marks_conf = []
        participant_count += 1
        nTrial = 0

        for dat in data_dict['data']:
            if dat['trialdata']['phase'] == 'TEST':
                nTrial+=1
                trial_data = {}
                trial_data['workerID'] = data_dict['workerId']
                trial_data['workerData'] = data_dict['questiondata']
                trial_data['trialData'] = dat['trialdata']


#                 trials_dict['Trial_'+str(count)] = trial_data
#                 trial_data = trial_data['trialData']
                trial_data['trialData']['correct response'] = get_gt_from_pagename(trial_data['trialData']['page_name'])
                trial_data['trialData']['choose response'] = trial_data['trialData']['Aclass']

#                 print(trial_data)
                if trial_data['trialData']['correct response'] == 'Catch':
                    catch_trial_mat[0][label_dict[trial_data['trialData']['choose response']]] += 1
                    catch_trial_mat_overall[0][label_dict[trial_data['trialData']['choose response']]] += 1
                elif trial_data['trialData']['correct response'] == 'Swapped':
                    swap_trial_mat[0][label_dict[trial_data['trialData']['choose response']]] += 1
                    swap_trial_mat_overall[0][label_dict[trial_data['trialData']['choose response']]] += 1
                else:
                    confmatrices['h_machine'][label_dict[trial_data['trialData']['correct response']]][label_dict[trial_data['trialData']['choose response']]] += 1
                    confmat_overall['h_machine'][label_dict[trial_data['trialData']['correct response']]][label_dict[trial_data['trialData']['choose response']]] += 1
                    if trial_data['trialData']['correct response'] == "Machine":
                        model = get_model_from_pagename(trial_data['trialData']['page_name'])
                        confmatrices['h_{}'.format(model)][0][label_dict[trial_data['trialData']['choose response']]] += 1
                        confmat_overall['h_{}'.format(model)][0][label_dict[trial_data['trialData']['choose response']]] += 1
                    marks.append(1 if trial_data['trialData']['correct response'] == trial_data['trialData']['choose response'] else 0)
                    marks_conf.append(deepcopy(confmatrices['h_machine']))

#                 trial_data['trialData'] = dat['trialdata']
                if trial_data['trialData']['correct response']  in ['Catch','Swapped']:
                    continue

                count += 1
                trial_data['trialData']['response_gender'] = trial_data['trialData']['Agender']
                del trial_data['trialData']['Agender']

                trial_data['trialData']['response_speaker'] = trial_data['trialData']['choose response'].lower()
                del trial_data['trialData']['choose response']

                trial_data['trialData']['response_object'] = trial_data['trialData']['topic'].lower()
                del trial_data['trialData']['topic']

                trial_data['trialData']['groundtruth'] = trial_data['trialData']['correct response'].lower()


                trial_data['trialData']['object_groundtruth'] = trial_data['trialData']['oneWord_curr'].lower()
                del trial_data['trialData']['oneWord_curr']

                if trial_data['trialData']['correct response'] in ["Machine","Catch","Swapped"]:
                    trial_data['trialData']['machine_groundtruth'] = modelname_dict[model.lower()]
                else:
                    trial_data['trialData']['machine_groundtruth'] = ''

                del trial_data['trialData']['correct response']
#                 trial_data['trialData']["imgtype"] = "naturaldesign"



                trials_dict['Trial_Obj_'+str(count)] = trial_data
#         plot_learning_curve_single_subject(marks_conf,participant_count)
        print(confmatrices)
        print('catch',catch_trial_mat)
#         print('swap',swap_trial_mat)
#     else:
#         print(all_data[i][15])
conn.close()

[Finished Date] 2025-04-22 09:58:20
[Time Used] 15.7 mins
[Questionnaire] {'country': 'United States', 'age': '41-45', 'engagement': '8', 'difficulty': '3', 'mode': 'live', 'gender': 'male', 'education': 'bachelor', 'ai-experience': '1', 'condition': 0, 'native': 'yes'}
[Counterbalance] 0
[Dummy Test Acc] 46/50
{'h_machine': array([[17.,  8.],
       [19.,  6.]]), 'h_azure': array([[3., 2.]]), 'h_detectron': array([[5., 0.]]), 'h_google': array([[4., 1.]]), 'h_rekognition': array([[3., 2.]]), 'h_chatgpt': array([[4., 1.]])}
catch [[0. 0.]]
[Finished Date] 2025-04-22 09:55:28
[Time Used] 9.8 mins
[Questionnaire] {'country': 'United States', 'age': '51', 'engagement': '7', 'difficulty': '10', 'mode': 'live', 'gender': 'female', 'education': 'master', 'ai-experience': '2', 'condition': 0, 'native': 'yes'}
[Counterbalance] 0
[Dummy Test Acc] 47/50
{'h_machine': array([[16.,  9.],
       [14., 11.]]), 'h_azure': array([[2., 3.]]), 'h_detectron': array([[3., 2.]]), 'h_google': array([[3., 2.

In [ ]:
confmat_overall

{'h_machine': array([[716., 534.],
        [481., 769.]]),
 'h_azure': array([[ 64., 186.]]),
 'h_detectron': array([[118., 132.]]),
 'h_google': array([[ 75., 175.]]),
 'h_rekognition': array([[ 53., 197.]]),
 'h_chatgpt': array([[171.,  79.]])}

In [5]:
np.mean(marker_overall)/50

np.float64(0.9124)

In [ ]:
trials_dict

{'Trial_Obj_1': {'workerID': 'A183WYXN12P2TJ',
  'workerData': {'country': 'United States',
   'age': '41-45',
   'engagement': '8',
   'difficulty': '3',
   'mode': 'live',
   'gender': 'male',
   'education': 'bachelor',
   'ai-experience': '1',
   'condition': 0,
   'native': 'yes'},
  'trialData': {'rt': 7441,
   'Aclass': 'Human',
   'hit': '10',
   'imageID': 'https://xiaoturing.s3.us-east-1.amazonaws.com/obj_det/394.jpg',
   'trial': 1,
   'page_name': '394_a_0.html/',
   'phase': 'TEST',
   'response_gender': 'NA',
   'response_speaker': 'human',
   'response_object': '1',
   'groundtruth': 'human',
   'object_groundtruth': '0',
   'machine_groundtruth': ''}},
 'Trial_Obj_2': {'workerID': 'A183WYXN12P2TJ',
  'workerData': {'country': 'United States',
   'age': '41-45',
   'engagement': '8',
   'difficulty': '3',
   'mode': 'live',
   'gender': 'male',
   'education': 'bachelor',
   'ai-experience': '1',
   'condition': 0,
   'native': 'yes'},
  'trialData': {'rt': 9993,
   'Acl

In [ ]:
len(trials_dict)

2500

In [ ]:
with open("/home/liuxiao/TuringGithubRunnable/multi_label_prediction/Plot/Data_all/humanjudge_Object_ChatGPT_testphase.json", 'w') as f:
    json.dump(trials_dict,f)

Bad pipe message: %s [b'><\x9f\x8eU\xde\n\xbb\xbf\x8d\xabZc\xc4u`8\xe4\x00\x02\xbc\x00\x00\x00\x01\x00\x02\x00\x03\x00\x04\x00\x05\x00\x06\x00\x07\x00\x08\x00\t\x00\n\x00\x0b\x00\x0c\x00\r\x00\x0e\x00\x0f\x00\x10\x00\x11\x00\x12\x00\x13\x00\x14\x00\x15\x00\x16\x00\x17\x00\x18\x00\x19\x00\x1a\x00\x1b\x00\x1e\x00\x1f\x00 \x00!\x00"\x00#\x00$\x00%\x00&\x00\'\x00(\x00)\x00']
Bad pipe message: %s [b'+\x00,\x00-\x00.\x00/\x000\x001\x002\x003\x004\x005\x006\x007\x008\x009\x00:\x00;\x00<\x00=\x00>\x00?']
Bad pipe message: %s [b'p\xe5MT\x0c\xae\x05\x9d\x0e"v\x03_\xb7\x18\xd5\x96\xb4\x00\x02\xbc\x00\x00\x00\x01\x00\x02\x00\x03\x00\x04\x00']
Bad pipe message: %s [b'\x06\x00\x07\x00']
Bad pipe message: %s [b'\t\x00\n\x00\x0b\x00\x0c']
Bad pipe message: %s [b'6\x9a(\xe3\xe5k\xc5\x90\x0b\xf2[\xff(:\x14R\x96\xa9\x00\x02\xbc\x00\x00\x00\x01\x00\x02\x00\x03\x00\x04\x00\x05\x00\x06\x00\x07\x00\x08\x00\t\x00\n\x00\x0b\x00\x0c\x00\r\x00\x0e\x00\x0f\x00\x10\x00\x11\x00\x12\x00\x13\x00\x14\x00\x15\x00\x16\x